# Tutorial 1: Token Operations

**Level:** Basic  
**Time:** 15-20 minutes  
**Prerequisites:** NeuroGraph installed and running

## Overview

This tutorial covers fundamental token operations in NeuroGraph:
- Creating tokens with 8D coordinates
- Reading and updating token properties
- Understanding token attributes (position, radius, weight)
- Managing token lifecycle (CRUD operations)

## What are Tokens?

Tokens are the fundamental building blocks of NeuroGraph's cognitive architecture. Each token represents a point in 8-dimensional space with:
- **Position**: 8D coordinates `[d0, d1, d2, d3, d4, d5, d6, d7]`
- **Radius**: Influence radius for spatial queries
- **Weight**: Importance factor (affects field calculations)

## Setup

In [ ]:
import requests
import json
from pprint import pprint

# API configuration
BASE_URL = "http://localhost:8000/api/v1"
headers = {"Content-Type": "application/json"}

# Helper function for pretty printing
def show(response):
    print(f"Status: {response.status_code}")
    pprint(response.json())

## Step 1: Authentication

First, we need to authenticate and get an access token.

In [ ]:
# Login
login_data = {
    "username": "admin",
    "password": "admin"
}

response = requests.post(f"{BASE_URL}/auth/login", json=login_data)
auth_data = response.json()

# Store access token
access_token = auth_data["access_token"]
headers["Authorization"] = f"Bearer {access_token}"

print(f"✓ Authenticated as {auth_data['user']['username']}")
print(f"Token expires in: {auth_data['expires_in']} seconds")

## Step 2: Create Your First Token

Let's create a token at the origin (all zeros) with default properties.

In [ ]:
# Create a token at origin
token_data = {
    "position": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    "radius": 1.0,
    "weight": 1.0
}

response = requests.post(f"{BASE_URL}/tokens", json=token_data, headers=headers)
token1 = response.json()

print("✓ Token created!")
show(response)

## Step 3: Create Multiple Tokens

Let's create a few more tokens at different positions.

In [ ]:
# Create tokens at different positions
tokens_to_create = [
    {"position": [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "radius": 1.5, "weight": 2.0},
    {"position": [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "radius": 0.8, "weight": 1.5},
    {"position": [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "radius": 2.0, "weight": 3.0},
    {"position": [2.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], "radius": 1.0, "weight": 1.0},
]

created_tokens = []
for token_data in tokens_to_create:
    response = requests.post(f"{BASE_URL}/tokens", json=token_data, headers=headers)
    created_tokens.append(response.json())

print(f"✓ Created {len(created_tokens)} tokens")
for token in created_tokens:
    print(f"  - Token {token['token_id']}: position={token['position'][:3]}... radius={token['radius']}")

## Step 4: List All Tokens

Retrieve all tokens in the system.

In [ ]:
# Get all tokens
response = requests.get(f"{BASE_URL}/tokens", headers=headers)
all_tokens = response.json()

print(f"✓ Total tokens: {len(all_tokens)}")
print("\nFirst 3 tokens:")
for token in all_tokens[:3]:
    print(f"  ID: {token['token_id']}")
    print(f"  Position: {token['position']}")
    print(f"  Radius: {token['radius']}, Weight: {token['weight']}")
    print()

## Step 5: Get a Specific Token

Retrieve a single token by ID.

In [ ]:
# Get first token by ID
token_id = all_tokens[0]['token_id']
response = requests.get(f"{BASE_URL}/tokens/{token_id}", headers=headers)

print(f"✓ Retrieved token {token_id}:")
show(response)

## Step 6: Update Token Properties

Modify token attributes like position, radius, or weight.

In [ ]:
# Update token - change radius and weight
update_data = {
    "radius": 2.5,
    "weight": 5.0
}

response = requests.put(f"{BASE_URL}/tokens/{token_id}", json=update_data, headers=headers)

print(f"✓ Updated token {token_id}:")
show(response)

## Step 7: Update Token Position

Move a token in 8D space.

In [ ]:
# Move token to new position
new_position = {
    "position": [5.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

response = requests.put(f"{BASE_URL}/tokens/{token_id}", json=new_position, headers=headers)

print(f"✓ Moved token {token_id} to new position:")
print(f"  Old: [0.0, 0.0, 0.0, ...]")
print(f"  New: {response.json()['position']}")

## Step 8: Delete a Token

Remove a token from the system.

In [ ]:
# Delete the last created token
token_to_delete = created_tokens[-1]['token_id']

response = requests.delete(f"{BASE_URL}/tokens/{token_to_delete}", headers=headers)

print(f"✓ Deleted token {token_to_delete}")
print(f"Status: {response.status_code}")

# Verify deletion
response = requests.get(f"{BASE_URL}/tokens", headers=headers)
print(f"\nRemaining tokens: {len(response.json())}")

## Step 9: Batch Operations

Create multiple tokens efficiently.

In [ ]:
# Create 10 tokens in a line
print("Creating 10 tokens in a line...")

line_tokens = []
for i in range(10):
    token_data = {
        "position": [float(i), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        "radius": 0.5,
        "weight": 1.0
    }
    response = requests.post(f"{BASE_URL}/tokens", json=token_data, headers=headers)
    line_tokens.append(response.json())

print(f"✓ Created {len(line_tokens)} tokens in a line")
print("\nToken positions:")
for token in line_tokens:
    print(f"  Token {token['token_id']}: x={token['position'][0]:.1f}")

## Step 10: Cleanup

Delete all tokens we created (optional).

In [ ]:
# Get all tokens and delete them
response = requests.get(f"{BASE_URL}/tokens", headers=headers)
all_tokens = response.json()

deleted_count = 0
for token in all_tokens:
    response = requests.delete(f"{BASE_URL}/tokens/{token['token_id']}", headers=headers)
    if response.status_code == 200:
        deleted_count += 1

print(f"✓ Deleted {deleted_count} tokens")

# Verify
response = requests.get(f"{BASE_URL}/tokens", headers=headers)
print(f"Remaining tokens: {len(response.json())}")

## Summary

In this tutorial, you learned:

✅ **Authentication** - Login and get access tokens  
✅ **Create tokens** - Define 8D positions, radius, and weight  
✅ **Read tokens** - List all tokens and get by ID  
✅ **Update tokens** - Modify properties and positions  
✅ **Delete tokens** - Remove tokens from the system  
✅ **Batch operations** - Create multiple tokens efficiently  

## Next Steps

- **Tutorial 2**: Grid Queries - Learn spatial search and neighbor finding
- **Tutorial 3**: CDNA Profiles - Configure dimensional behavior
- **Intermediate Tutorials**: WebSocket events, REST API integration

## Key Takeaways

1. **Tokens are spatial entities** in 8D space
2. **Position, radius, weight** define token behavior
3. **REST API** provides full CRUD operations
4. **Authentication required** for all operations
5. **Batch operations** possible for efficiency

---

**Need help?** Check the [API Reference](https://your-docs-url/api/index.html) or [Quick Start Guide](https://your-docs-url/quickstart.html)